In [1]:
import pandas as pd
import numpy as np
import uuid
import csv
from datetime import datetime, timedelta

# ==========================================
# 1. CONFIGURACIÓN Y MAESTROS (INTACTOS)
# ==========================================
num_rows_target = 2100000
output_file = 'data_clase_sql_final.csv'
np.random.seed(42)

paises_config = {
    'México': {'share': 0.40, 'mult': 0.90},
    'España': {'share': 0.25, 'mult': 1.25},
    'Colombia': {'share': 0.15, 'mult': 0.85},
    'Argentina': {'share': 0.12, 'mult': 1.00},
    'Chile': {'share': 0.08, 'mult': 1.10}
}

usuarios_pool = np.arange(10000, 99999)
user_geo = {u: np.random.choice(list(paises_config.keys()), p=[c['share'] for c in paises_config.values()]) for u in usuarios_pool}
skus_pool = [f"SKU-{i}" for i in range(100, 1000)]

print(f"Generando {num_rows_target:,} registros con embudo progresivo...")

# ==========================================
# 2. GENERACIÓN CON FRICCIÓN Y END_CHECKOUT
# ==========================================
headers = ['event_timestamp', 'user_id', 'session_id', 'event_name', 'country', 'traffic_source', 'device', 'sku', 'cart_id', 'payment_method', 'units', 'amount']

with open(output_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    
    current_rows = 0
    while current_rows < num_rows_target:
        uid = int(np.random.choice(usuarios_pool))
        pais, mult = user_geo[uid], paises_config[user_geo[uid]]['mult']
        sid = str(uuid.uuid4())[:18].upper()
        ts = datetime(2026, 1, 1) + timedelta(seconds=np.random.randint(0, 2678400))
        
        src = np.random.choice(['Google Search', 'Direct', 'Social Media', 'Email Campaign'], p=[0.25, 0.20, 0.50, 0.05])
        dev = np.random.choice(['Mobile', 'Desktop', 'Tablet'], p=[0.6, 0.3, 0.1])
        
        session_rows = []
        # PASO 1: Home (100% de las sesiones entran aquí)
        session_rows.append([ts.strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'home_page', pais, src, dev, None, None, None, 0, 0.0])
        
        # PASO 2: Login (Fricción: 75%)
        if np.random.random() < 0.75:
            ts += timedelta(seconds=20)
            session_rows.append([ts.strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'login', pais, src, dev, None, None, None, 0, 0.0])
            
            # PASO 3: Product View (Fricción: 80% de los logueados ven productos)
            if np.random.random() < 0.80:
                num_views = np.random.randint(1, 4)
                for _ in range(num_views):
                    ts += timedelta(minutes=2)
                    session_rows.append([ts.strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'product_view', pais, src, dev, np.random.choice(skus_pool), None, None, 0, 0.0])
                
                # PASO 4: Add to Cart (Fricción: 40% de los que ven, agregan al carrito)
                conv_base = 0.60 if src == 'Email Campaign' else 0.35
                if np.random.random() < conv_base:
                    cart_id_fijo = f"CART-{np.random.randint(10000, 99999)}"
                    pago_fijo = np.random.choice(['Credit Card', 'PayPal', 'Debit Card', 'Crypto'])
                    num_art = np.random.randint(1, 3)
                    skus_compra = np.random.choice(skus_pool, size=num_art, replace=False)
                    
                    for sku in skus_compra:
                        ts_step = ts + timedelta(seconds=30)
                        session_rows.append([ts_step.strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'add_to_cart', pais, src, dev, sku, cart_id_fijo, None, 0, 0.0])
                        
                        # PASO 5: Begin Checkout (Fricción: 85% de los que agregan)
                        if np.random.random() < 0.85:
                            session_rows.append([(ts_step + timedelta(seconds=10)).strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'begin_checkout', pais, src, dev, sku, cart_id_fijo, None, 0, 0.0])
                            
                            # PASO 6: End Checkout (Fricción: 90% de los que inician)
                            if np.random.random() < 0.90:
                                session_rows.append([(ts_step + timedelta(seconds=20)).strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'end_checkout', pais, src, dev, sku, cart_id_fijo, None, 0, 0.0])
                                
                                # PASO 7: Purchase (Fricción: 95% de los que terminan el proceso)
                                if np.random.random() < 0.95:
                                    u, a = np.random.randint(1, 3), round(np.random.uniform(40, 150) * mult, 2)
                                    session_rows.append([(ts_step + timedelta(seconds=30)).strftime('%Y-%m-%d %H:%M:%S'), uid, sid, 'purchase', pais, src, dev, sku, cart_id_fijo, pago_fijo, u, a])

        for r in session_rows:
            writer.writerow(r)
        current_rows += len(session_rows)

# ==========================================
# 3. FIX FORZADO Y GENERACIÓN DF_FINAL
# ==========================================
print("Ejecutando saneamiento de integridad y cargando df_final...")
df_final = pd.read_csv(output_file)

# El Seguro: Forzar el primer pago por cada Cart ID
fix_pago = df_final[df_final['event_name'] == 'purchase'].groupby('cart_id')['payment_method'].first()
df_final.loc[df_final['event_name'] == 'purchase', 'payment_method'] = df_final.loc[df_final['event_name'] == 'purchase', 'cart_id'].map(fix_pago)

# Guardar y Validar
#df_final.to_csv(output_file, index=False)

# ==========================================
# 4. REPORTE FINAL DE CALIDAD
# ==========================================
print("\n" + "="*40)
print("EMBUDO DE CONVERSIÓN ACTUALIZADO:")
print(df_final['event_name'].value_counts())
print("\nAUDITORÍA DE PAGOS:")
errores = df_final[df_final['event_name'] == 'purchase'].groupby('cart_id')['payment_method'].nunique().gt(1).sum()
print(f"Errores de pago: {errores}")
print("="*40)

Generando 2,100,000 registros con embudo progresivo...
Ejecutando saneamiento de integridad y cargando df_final...

EMBUDO DE CONVERSIÓN ACTUALIZADO:
event_name
product_view      623316
home_page         519787
login             390101
add_to_cart       169535
begin_checkout    144197
end_checkout      129890
purchase          123174
Name: count, dtype: int64

AUDITORÍA DE PAGOS:
Errores de pago: 0


In [3]:
usuarios_unicos_por_evento = df_final.groupby('event_name')['user_id'].nunique().sort_values(ascending=False)

print(usuarios_unicos_por_evento)

event_name
home_page         89735
login             88848
product_view      87184
add_to_cart       64474
begin_checkout    61550
end_checkout      59382
purchase          58163
Name: user_id, dtype: int64


In [9]:
df_final.shape

(2100000, 12)

In [10]:
df_final[df_final['event_name'] == 'begin_checkout']

,event_timestamp,user_id,session_id,event_name,country,traffic_source,device,sku,cart_id,payment_method,units,amount
12,2026-01-20 04:15:18,66728,F6EB03E8-8F39-4459,begin_checkout,Colombia,Direct,Mobile,SKU-187,CART-71439,NaN,0,0.0
31,2026-01-23 14:39:32,24840,B6B03CE5-908F-444A,begin_checkout,México,Social Media,Mobile,SKU-881,CART-64624,NaN,0,0.0
45,2026-01-19 03:25:17,59137,14FF9FFA-C492-4DBE,begin_checkout,Argentina,Social Media,Desktop,SKU-932,CART-40090,NaN,0,0.0
56,2026-01-12 03:17:43,57051,1D3A3DAC-2FA4-45E5,begin_checkout,Colombia,Email Campaign,Mobile,SKU-932,CART-63546,NaN,0,0.0
72,2026-01-16 19:52:21,82720,1443DA21-010B-4D77,begin_checkout,México,Google Search,Tablet,SKU-366,CART-26211,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2099925,2026-01-09 07:14:54,69908,5914E8FE-4CE0-4C1C,begin_checkout,España,Social Media,Mobile,SKU-686,CART-59277,NaN,0,0.0
2099933,2026-01-03 07:11:14,17496,41E2B15F-451B-4941,begin_checkout,México,Social Media,Mobile,SKU-244,CART-13979,NaN,0,0.0
2099937,2026-01-03 07:11:14,17496,41E2B15F-451B-4941,begin_checkout,México,Social Media,Mobile,SKU-280,CART-13979,NaN,0,0.0
2099950,2026-01-21 13:14:00,56319,2F8DC2E5-D41A-47B3,begin_checkout,España,Direct,Mobile,SKU-999,CART-17015,NaN,0,0.0


In [11]:
import pandas as pd
import numpy as np

def validate_ecommerce_data(file_path):
    print(f"--- Iniciando Auditoría de Coherencia para: {file_path} ---\n")
    df = pd.read_csv(file_path)
    
    # 1. Validación de Montos (Solo en compras)
    check_amount = df[(df['event_name'] != 'purchase') & (df['amount'] > 0)]
    val1 = "PASSED" if len(check_amount) == 0 else f"FAILED ({len(check_amount)} errores)"
    print(f"1. Montos fuera de evento 'purchase': {val1}")

    # 2. Validación de Unicidad de Session ID
    # Verificamos que un session_id pertenezca a un solo user_id
    session_user_count = df.groupby('session_id')['user_id'].nunique()
    val2 = "PASSED" if (session_user_count == 1).all() else "FAILED (Sesiones compartidas entre usuarios)"
    print(f"2. Integridad Session-User (1:1): {val2}")

    # 3. Validación de Proporción Poblacional
    # México debe ser > España > Chile
    country_counts = df['country'].value_counts()
    is_proportional = (country_counts['México'] > country_counts['España']) and \
                      (country_counts['España'] > country_counts['Chile'])
    val3 = "PASSED" if is_proportional else "FAILED (Distribución plana detectada)"
    print(f"3. Jerarquía Poblacional Realista: {val3}")

    # 4. Validación de Unidades
    check_units = df[(df['event_name'] == 'purchase') & (df['units'] <= 0)]
    val4 = "PASSED" if len(check_units) == 0 else f"FAILED ({len(check_units)} compras sin unidades)"
    print(f"4. Unidades mínimas en compras (>0): {val4}")

    

    # 6. Validación de Sesión vs Login
    # Toda compra debería haber tenido un evento de 'login' previo en la misma sesión
    purchases_sessions = df[df['event_name'] == 'purchase']['session_id'].unique()
    sessions_with_login = df[df['event_name'] == 'login']['session_id'].unique()
    missing_login = set(purchases_sessions) - set(sessions_with_login)
    val6 = "PASSED" if len(missing_login) == 0 else f"FAILED ({len(missing_login)} sesiones compraron sin login)"
    print(f"6. Flujo de Login antes de compra: {val6}")

    # 7. Validación de Product Views por Sesión (Campana de Gauss)
    avg_views = df[df['event_name'] == 'product_view'].groupby('session_id').size().mean()
    val7 = "PASSED" if 3.5 <= avg_views <= 4.5 else f"FAILED (Media: {avg_views:.2f})"
    print(f"7. Distribución Normal de Views (~4): {val7}")

    # 8. Validación de Valores Nulos (Missing Values)
    critical_cols = ['event_timestamp', 'user_id', 'session_id', 'event_name', 'country']
    nulls = df[critical_cols].isnull().sum().sum()
    val8 = "PASSED" if nulls == 0 else f"FAILED ({nulls} nulos detectados)"
    print(f"8. Integridad de Columnas Críticas (Not Null): {val8}")

    # 9. Validación de Cart ID
    # Verificamos que el cart_id esté presente en eventos de checkout y purchase
    cart_check = df[df['event_name'].isin(['add_to_cart', 'purchase'])]['cart_id'].isnull().sum()
    val9 = "PASSED" if cart_check == 0 else "FAILED (Eventos de carrito sin Cart ID)"
    print(f"9. Consistencia de Cart ID en el funnel: {val9}")

    # 10. Validación de Formato de Tiempo
    try:
        pd.to_datetime(df['event_timestamp'])
        val10 = "PASSED"
    except:
        val10 = "FAILED (Formato de fecha inválido)"
    print(f"10. Integridad de Event Timestamp: {val10}")

    print("\n--- Auditoría Finalizada ---")

# Ejecución
validate_ecommerce_data('data_clase_sql_final.csv')

--- Iniciando Auditoría de Coherencia para: data_clase_sql_final.csv ---

1. Montos fuera de evento 'purchase': PASSED
2. Integridad Session-User (1:1): PASSED
3. Jerarquía Poblacional Realista: PASSED
4. Unidades mínimas en compras (>0): PASSED
6. Flujo de Login antes de compra: PASSED
7. Distribución Normal de Views (~4): FAILED (Media: 2.00)
8. Integridad de Columnas Críticas (Not Null): PASSED
9. Consistencia de Cart ID en el funnel: PASSED
10. Integridad de Event Timestamp: PASSED

--- Auditoría Finalizada ---


In [12]:
import pandas as pd

def generar_tablas_analisis(file_path):
    df = pd.read_csv(file_path)
    
    # Pre-cálculo de órdenes (solo eventos de compra)
    df_compras = df[df['event_name'] == 'purchase']

    # --- TABLA 1: MÉTRICAS POR PAÍS (Validación de Población y Valor) ---
    print("\n1. TABLA POR PAÍS (Volumen y Negocio)")
    tabla_pais = df.groupby('country').agg(
        total_eventos=('event_name', 'count'),
        usuarios_unicos=('user_id', 'nunique'),
        sesiones_unicos=('session_id', 'nunique'),
        total_ordenes=('amount', lambda x: (x > 0).sum()),
        ingresos_totales=('amount', 'sum')
    ).sort_values('total_eventos', ascending=False)
    print(tabla_pais)

    # --- TABLA 2: MÉTRICAS POR DISPOSITIVO (Validación de Tecnología) ---
    print("\n2. TABLA POR DISPOSITIVO")
    tabla_device = df.groupby('device').agg(
        total_eventos=('event_name', 'count'),
        sesiones=('session_id', 'nunique'),
        compras=('event_name', lambda x: (x == 'purchase').sum())
    )
    print(tabla_device)

    # --- TABLA 3: SESGO DE TRÁFICO (Validación de Canales) ---
    print("\n3. TABLA POR FUENTE DE TRÁFICO (Efectividad)")
    tabla_source = df.groupby('traffic_source').agg(
        total_eventos=('event_name', 'count'),
        compras=('event_name', lambda x: (x == 'purchase').sum()),
        ticket_promedio=('amount', lambda x: x[x > 0].mean())
    )
    # Calculamos Tasa de Conversión (Compras / Sesiones Únicas de esa fuente)
    tabla_source['conversion_rate'] = (tabla_source['compras'] / df.groupby('traffic_source')['session_id'].nunique() * 100).round(2)
    print(tabla_source.sort_values('conversion_rate', ascending=False))

    # --- TABLA 4: DISTRIBUCIÓN DE EVENTOS (Validación del Funnel) ---
    print("\n4. CONTEO GLOBAL DE EVENTOS (Embudo)")
    tabla_eventos = df['event_name'].value_counts().to_frame()
    print(tabla_eventos)

# Ejecución
generar_tablas_analisis('data_clase_sql_final.csv')


1. TABLA POR PAÍS (Volumen y Negocio)
           total_eventos  usuarios_unicos  sesiones_unicos  total_ordenes  \
country                                                                     
México            840195            35828           207995          49269   
España            525429            22475           130001          30945   
Colombia          315815            13500            78143          18547   
Argentina         253746            10845            62786          14810   
Chile             164815             7087            40862           9603   

           ingresos_totales  
country                      
México           4212361.55  
España           3675587.45  
Colombia         1493575.39  
Argentina        1406451.23  
Chile            1004787.71  

2. TABLA POR DISPOSITIVO
         total_eventos  sesiones  compras
device                                   
Desktop         629141    155674    37005
Mobile         1260453    312012    73845
Tablet          2

In [13]:
import pandas as pd

def analizar_compras_multiples(file_path):
    print(f"--- Análisis de Órdenes con Múltiples Productos ---")
    df = pd.read_csv(file_path)
    
    # 1. Filtramos solo los eventos de compra
    compras = df[df['event_name'] == 'purchase'].copy()
    
    # 2. Agrupamos por Sesión y Usuario para consolidar la orden
    ordenes = compras.groupby(['session_id', 'user_id']).agg(
        cantidad_skus_distintos=('sku', 'nunique'),
        lista_productos=('sku', lambda x: list(x)),
        total_invertido=('amount', 'sum'),
        total_unidades=('units', 'sum')
    ).reset_index()
    
    # 3. Filtramos los casos donde compraron más de un SKU distinto
    compras_multiples = ordenes[ordenes['cantidad_skus_distintos'] > 1].sort_values(by='cantidad_skus_distintos', ascending=False)
    
    if not compras_multiples.empty:
        print(f"ÉXITO: Se encontraron {len(compras_multiples):,} sesiones con compra de múltiples productos.")
        print("\nTop 10 Casos Detectados (Muestra):")
        # Ajustamos el formato de visualización para ver bien las listas
        pd.set_option('display.max_colwidth', None)
        print(compras_multiples[['user_id', 'session_id', 'cantidad_skus_distintos', 'lista_productos', 'total_invertido']].head(10))
        
        # 4. Estadísticas rápidas para validación de sesgos
        avg_items = ordenes['cantidad_skus_distintos'].mean()
        print(f"\nEstadística Global:")
        print(f"- Promedio de SKUs distintos por orden: {avg_items:.2f}")
    else:
        print("ALERTA: No se detectaron compras de múltiples SKUs. Revisar lógica del generador.")

# Ejecución de la función
analizar_compras_multiples('data_clase_sql_final.csv')

--- Análisis de Órdenes con Múltiples Productos ---
ÉXITO: Se encontraron 29,796 sesiones con compra de múltiples productos.

Top 10 Casos Detectados (Muestra):
       user_id          session_id  cantidad_skus_distintos  \
3        39571  00020A46-B9E8-44D8                        2   
62043    66403  A90A939D-8BD4-4D10                        2   
62099    27346  A9383EB7-9614-41EA                        2   
62090    51694  A92CF9D0-0053-43B9                        2   
62088    28266  A92C7F5C-B852-4CEB                        2   
62087    28030  A92B800B-2069-461F                        2   
62080    39882  A92500D5-B07B-490F                        2   
62079    63756  A9245C8D-04A5-4791                        2   
62075    75841  A921F3BB-A654-4232                        2   
62073    80672  A921AAA4-5452-47E8                        2   

          lista_productos  total_invertido  
3      [SKU-268, SKU-166]           174.72  
62043  [SKU-477, SKU-124]           137.61  
62099  [SK

In [14]:
import pandas as pd

def validar_compras_multiples(file_path):
    print(f"--- Auditoría de Canastas Multi-Producto ---")
    df = pd.read_csv(file_path)
    
    # 1. Filtramos solo los eventos de compra
    compras = df[df['event_name'] == 'purchase'].copy()
    
    # 2. Agrupamos por usuario y sesión para ver la "Cesta"
    # Concatenamos los SKUs y sumamos montos/unidades
    cestas = compras.groupby(['user_id', 'session_id', 'country']).agg(
        distintos_skus=('sku', 'nunique'),
        lista_skus=('sku', lambda x: list(x)),
        total_unidades=('units', 'sum'),
        total_pagado=('amount', 'sum')
    ).reset_index()
    
    # 3. Filtramos los casos donde compraron 2 o más SKUs diferentes
    multi_sku = cestas[cestas['distintos_skus'] > 1]
    if not multi_sku.empty:
        print(f"ÉXITO: Se encontraron {len(multi_sku):,} sesiones con múltiples productos.")
        print("\nMuestra de los Top 10 casos detectados:")
        # Ajuste para ver la lista completa de SKUs
        pd.set_option('display.max_colwidth', None)
        print(multi_sku[['user_id', 'country', 'distintos_skus', 'lista_skus', 'total_pagado']].head(50))
        
        # Validación de Identidad (Victoria previa)
        print(f"\n--- Verificación de Consistencia Geográfica ---")
        conteo_paises = multi_sku.groupby('user_id')['country'].nunique()
        if conteo_paises.max() == 1:
            print("CONFIRMADO: Cada usuario mantiene un único país en sus compras múltiples.")
    else:
        print("ALERTA: No se detectaron compras múltiples. Revisar lógica del generador.")

# Ejecución
validar_compras_multiples('data_clase_sql_final.csv')

--- Auditoría de Canastas Multi-Producto ---
ÉXITO: Se encontraron 29,796 sesiones con múltiples productos.

Muestra de los Top 10 casos detectados:
     user_id    country  distintos_skus          lista_skus  total_pagado
2      10002   Colombia               2  [SKU-908, SKU-759]        137.34
4      10003     España               2  [SKU-882, SKU-125]        263.51
5      10004     México               2  [SKU-212, SKU-588]        143.85
6      10005     México               2  [SKU-737, SKU-980]        138.83
13     10008     España               2  [SKU-340, SKU-646]        197.19
15     10009   Colombia               2  [SKU-770, SKU-570]        207.06
16     10011      Chile               2  [SKU-264, SKU-518]        193.65
19     10019     México               2  [SKU-414, SKU-651]        193.79
22     10023     México               2  [SKU-367, SKU-862]         99.46
23     10026     México               2  [SKU-643, SKU-515]        176.13
25     10028     España              

In [16]:
df_final[(df_final['user_id'] == 10002)].sort_values(by='event_timestamp')

,event_timestamp,user_id,session_id,event_name,country,traffic_source,device,sku,cart_id,payment_method,units,amount
637500,2026-01-08 12:07:48,10002,F3E2B3D0-A0A1-4135,home_page,Colombia,Social Media,Tablet,NaN,NaN,NaN,0,0.00
637501,2026-01-08 12:08:08,10002,F3E2B3D0-A0A1-4135,login,Colombia,Social Media,Tablet,NaN,NaN,NaN,0,0.00
637502,2026-01-08 12:10:08,10002,F3E2B3D0-A0A1-4135,product_view,Colombia,Social Media,Tablet,SKU-215,NaN,NaN,0,0.00
637503,2026-01-08 12:12:08,10002,F3E2B3D0-A0A1-4135,product_view,Colombia,Social Media,Tablet,SKU-728,NaN,NaN,0,0.00
637508,2026-01-08 12:12:38,10002,F3E2B3D0-A0A1-4135,add_to_cart,Colombia,Social Media,Tablet,SKU-759,CART-27166,NaN,0,0.00
637504,2026-01-08 12:12:38,10002,F3E2B3D0-A0A1-4135,add_to_cart,Colombia,Social Media,Tablet,SKU-908,CART-27166,NaN,0,0.00
637509,2026-01-08 12:12:48,10002,F3E2B3D0-A0A1-4135,begin_checkout,Colombia,Social Media,Tablet,SKU-759,CART-27166,NaN,0,0.00
637505,2026-01-08 12:12:48,10002,F3E2B3D0-A0A1-4135,begin_checkout,Colombia,Social Media,Tablet,SKU-908,CART-27166,NaN,0,0.00
637506,2026-01-08 12:12:58,10002,F3E2B3D0-A0A1-4135,end_checkout,Colombia,Social Media,Tablet,SKU-908,CART-27166,NaN,0,0.00
637510,2026-01-08 12:12:58,10002,F3E2B3D0-A0A1-4135,end_checkout,Colombia,Social Media,Tablet,SKU-759,CART-27166,NaN,0,0.00


In [17]:
# 1. Contamos cuántos usuarios únicos llegaron a cada etapa
embudo_real = df_final.groupby('event_name')['user_id'].nunique().sort_values(ascending=False)

# 2. Lo convertimos a DataFrame para que sea más legible
df_embudo = embudo_real.to_frame(name='usuarios_unicos')

# 3. Calculamos el porcentaje de retención respecto al paso anterior
df_embudo['retencion_%'] = (df_embudo['usuarios_unicos'] / df_embudo['usuarios_unicos'].shift(1) * 100).round(2)

print(df_embudo)

                usuarios_unicos  retencion_%
event_name                                  
home_page                 89735          NaN
login                     88848        99.01
product_view              87184        98.13
add_to_cart               64474        73.95
begin_checkout            61550        95.46
end_checkout              59382        96.48
purchase                  58163        97.95


In [18]:
import pandas as pd

# 1. Cargamos el dataframe que acabas de generar
# Nota: Si ya tienes 'df_final' en memoria, puedes saltar la lectura
if 'df_final' not in locals():
    print("Cargando df_final desde el CSV...")
    df_final = pd.read_csv('data_clase_sql_final.csv')

# 2. Filtramos solo los eventos de compra
df_purchases = df_final[df_final['event_name'] == 'purchase']

# 3. Agrupamos por cart_id y contamos cuántos métodos de pago únicos tiene cada uno
check_pagos = df_purchases.groupby('cart_id')['payment_method'].nunique().reset_index()

# 4. Identificamos carritos que tengan más de 1 método (el error que teníamos)
carritos_con_error = check_pagos[check_pagos['payment_method'] > 1]

# 5. Output de control
print("-" * 40)
print(f"RESUMEN DE CALIDAD DE PAGOS")
print("-" * 40)
print(f"Total de carritos analizados: {len(check_pagos):,}")
print(f"Carritos con errores de consistencia: {len(carritos_con_error)}")

if len(carritos_con_error) > 0:
    print("\nDetalle de los primeros errores encontrados:")
    print(carritos_con_error.head())
else:
    print("\n✅ VALIDACIÓN EXITOSA: Cada cart_id tiene un único payment_method.")
print("-" * 40)

----------------------------------------
RESUMEN DE CALIDAD DE PAGOS
----------------------------------------
Total de carritos analizados: 58,181
Carritos con errores de consistencia: 0

✅ VALIDACIÓN EXITOSA: Cada cart_id tiene un único payment_method.
----------------------------------------


In [19]:
import pandas as pd

# 1. Aseguramos que df_final esté cargado con el último CSV generado
print("Cargando df_final para auditoría final...")
df_final = pd.read_csv('data_clase_sql_final.csv')

# 2. Validación: Consistencia de Pago por Carrito
# Solo analizamos eventos de 'purchase' donde el cart_id debe tener 1 solo método
audit_pagos = df_final[df_final['event_name'] == 'purchase'].groupby('cart_id')['payment_method'].nunique()
errores_pago = audit_pagos[audit_pagos > 1].count()

# 3. Validación: Integridad de Usuario (1 País por User_ID)
audit_paises = df_final.groupby('user_id')['country'].nunique()
errores_pais = audit_paises[audit_paises > 1].count()

# --- RESULTADOS ---
print("-" * 40)
print(f"REPORTES DE CONSISTENCIA")
print("-" * 40)
print(f"Errores de Pago (Mismo carrito con varios métodos): {errores_pago}")
print(f"Errores de País (Mismo usuario con varios países): {errores_pais}")
print("-" * 40)

if errores_pago == 0 and errores_pais == 0:
    print("✅ CALIDAD CERTIFICADA: El dataset es consistente.")
else:
    print("❌ ATENCIÓN: Se detectaron inconsistencias en la generación.")

Cargando df_final para auditoría final...
----------------------------------------
REPORTES DE CONSISTENCIA
----------------------------------------
Errores de Pago (Mismo carrito con varios métodos): 0
Errores de País (Mismo usuario con varios países): 0
----------------------------------------
✅ CALIDAD CERTIFICADA: El dataset es consistente.


In [32]:
# Cruzamos Fuente de Tráfico vs Dispositivo para ver la distribución
pivot_variabilidad = pd.crosstab(df_final['traffic_source'], df_final['device'], normalize='index') * 100

print("--- Distribución de Dispositivos por Fuente (%) ---")
print(pivot_variabilidad.round(2))

# Verificación de Variabilidad:
# Si todos los números fueran idénticos (ej. todos 60.00), la data sería sospechosa.
# Al haber decimales distintos (ej. 59.82 vs 60.15), confirmamos aleatoriedad orgánica.

--- Distribución de Dispositivos por Fuente (%) ---
device          Desktop  Mobile  Tablet
traffic_source                         
Direct            29.91   59.97   10.12
Email Campaign    29.66   60.19   10.14
Google Search     29.84   60.40    9.76
Social Media      30.08   59.83   10.09


### __Productos__

In [33]:
import pandas as pd
import numpy as np
import random

# 1. Definición de coherencia (Marca -> Categoría) y (Subcategoría -> Modelos)
category_brands = {
    'Electronics': ['Samsung', 'Apple', 'Sony', 'Xiaomi'],
    'Home & Kitchen': ['Dyson', 'Philips', 'Ninja', 'DeLonghi'],
    'Fashion': ['Nike', 'Adidas', 'Patagonia', 'Columbia'],
    'Beauty': ['L\'Oréal', 'Chanel', 'Estée Lauder']
}

catalog_details = {
    'Electronics': {
        'Smartphones': ['Galaxy S23', 'iPhone 15', 'Pixel 8', 'Redmi Note'],
        'Laptops': ['MacBook Air', 'XPS 13', 'ThinkPad', 'Surface'],
        'Headphones': ['WH-1000XM5', 'AirPods Pro', 'Buds Pro']
    },
    'Home & Kitchen': {
        'Coffee Makers': ['Vertuo', 'K-Elite', 'Magnifica'],
        'Air Fryers': ['Foodi', 'AirFryer XXL', 'Turbo Blaze'],
        'Vacuum Cleaners': ['V15 Detect', 'Roomba j7', 'Stratos']
    },
    'Fashion': {
        'Running Shoes': ['UltraBoost', 'Air Max', 'Clifton', 'Pegasus'],
        'Jackets': ['Nuptse', 'Ascender', 'Down Sweater'],
        'Watches': ['G-Shock', 'Seiko 5', 'Fenix']
    },
    'Beauty': {
        'Skincare': ['Hyaluronic Acid', 'Retinol Cream', 'Vitamin C'],
        'Fragrances': ['Sauvage', 'Chanel No. 5', 'Acqua di Gio']
    }
}

vendors_map = {
    'Electronics': 'TechLogistics Corp',
    'Home & Kitchen': 'Global Home Solutions',
    'Fashion': 'Style & Wear Co.',
    'Beauty': 'Beauty Essentials Ltd'
}

# 2. Construcción de la Dimensión con Coherencia Total
product_rows = []

for i in range(100, 1501):
    sku = f"SKU-{i}"
    # Elegimos categoría y luego marca/subcategoría que PERTENEZCAN a esa categoría
    cat = random.choice(list(catalog_details.keys()))
    brand = random.choice(category_brands[cat])
    sub_cat = random.choice(list(catalog_details[cat].keys()))
    model = random.choice(catalog_details[cat][sub_cat])
    
    # Atributos lógicos
    vendor = vendors_map[cat]
    weight = round(random.uniform(0.1, 5.0) if cat != 'Home & Kitchen' else random.uniform(2.0, 10.0), 2)
    rating = round(random.uniform(3.5, 5.0), 1)

    product_rows.append([
        sku, f"{brand} {model}", cat, sub_cat, brand, vendor, weight, rating
    ])

# 3. Crear DataFrame Final (Sin unit_price ni unit_cost)
df_products = pd.DataFrame(product_rows, columns=[
    'sku', 'product_name', 'category', 'sub_category', 'brand', 'vendor', 'weight_kg', 'product_rating'
])

# Guardar y regenerar df_final (solo para asegurar que lo tienes en memoria)
df_products.to_csv('dim_products.csv', index=False)

print("✅ dim_products.csv corregida:")
print("- Sin columnas de costo/precio.")
print("- Coherencia de marca y categoría total (Ej: Samsung solo en Electronics).")
print(f"Columnas actuales: {list(df_products.columns)}")

✅ dim_products.csv corregida:
- Sin columnas de costo/precio.
- Coherencia de marca y categoría total (Ej: Samsung solo en Electronics).
Columnas actuales: ['sku', 'product_name', 'category', 'sub_category', 'brand', 'vendor', 'weight_kg', 'product_rating']


In [34]:
df_products.head(20)

,sku,product_name,category,sub_category,brand,vendor,weight_kg,product_rating
0,SKU-100,Ninja Vertuo,Home & Kitchen,Coffee Makers,Ninja,Global Home Solutions,6.29,4.5
1,SKU-101,Nike UltraBoost,Fashion,Running Shoes,Nike,Style & Wear Co.,4.91,4.4
2,SKU-102,Samsung ThinkPad,Electronics,Laptops,Samsung,TechLogistics Corp,1.22,4.2
3,SKU-103,Ninja Turbo Blaze,Home & Kitchen,Air Fryers,Ninja,Global Home Solutions,6.28,4.2
4,SKU-104,Nike Nuptse,Fashion,Jackets,Nike,Style & Wear Co.,1.51,3.9
5,SKU-105,Patagonia Pegasus,Fashion,Running Shoes,Patagonia,Style & Wear Co.,2.36,4.9
6,SKU-106,Chanel Vitamin C,Beauty,Skincare,Chanel,Beauty Essentials Ltd,3.68,4.9
7,SKU-107,Ninja Foodi,Home & Kitchen,Air Fryers,Ninja,Global Home Solutions,8.77,3.7
8,SKU-108,Estée Lauder Hyaluronic Acid,Beauty,Skincare,Estée Lauder,Beauty Essentials Ltd,0.97,3.9
9,SKU-109,Xiaomi Surface,Electronics,Laptops,Xiaomi,TechLogistics Corp,2.88,4.7


In [35]:
# Unimos los hechos de compra con la dimensión de productos
df_analysis = df_final[df_final['event_name'] == 'purchase'].merge(
    df_products, 
    on='sku', 
    how='inner'
)

print(f"Total de registros de compra con información de producto: {len(df_analysis):,}")

Total de registros de compra con información de producto: 123,174


In [36]:
# Conteo de ventas por país y categoría
coherencia_cat = df_analysis.groupby(['country', 'category']).size().unstack(fill_value=0)

print("Distribución de Ventas por País y Categoría:")
print(coherencia_cat)

Distribución de Ventas por País y Categoría:
category   Beauty  Electronics  Fashion  Home & Kitchen
country                                                
Argentina    3452         4020     3806            3532
Chile        2216         2562     2476            2349
Colombia     4341         4903     4765            4538
España       7216         8175     8003            7551
México      11541        12983    12801           11944


In [37]:
# Top 5 marcas con más unidades vendidas por país
top_brands = df_analysis.groupby(['country', 'brand'])['units'].sum().reset_index()
top_brands = top_brands.sort_values(['country', 'units'], ascending=[True, False]).groupby('country').head(5)

print("Top 5 Marcas por Unidades Vendidas:")
print(top_brands)

Top 5 Marcas por Unidades Vendidas:
      country         brand  units
6   Argentina  Estée Lauder   1924
12  Argentina       Samsung   1737
2   Argentina        Chanel   1696
7   Argentina       L'Oréal   1548
14  Argentina        Xiaomi   1545
17      Chile        Chanel   1155
27      Chile       Samsung   1138
21      Chile  Estée Lauder   1132
29      Chile        Xiaomi   1035
22      Chile       L'Oréal   1034
36   Colombia  Estée Lauder   2388
42   Colombia       Samsung   2087
37   Colombia       L'Oréal   2047
32   Colombia        Chanel   2038
44   Colombia        Xiaomi   1926
51     España  Estée Lauder   3791
47     España        Chanel   3697
57     España       Samsung   3594
52     España       L'Oréal   3329
48     España      Columbia   3320
66     México  Estée Lauder   6204
62     México        Chanel   5855
72     México       Samsung   5621
67     México       L'Oréal   5282
63     México      Columbia   5251


In [38]:
# Correlación entre Rating y Unidades vendidas
popularity = df_analysis.groupby('product_name').agg({
    'units': 'sum',
    'product_rating': 'first',
    'category': 'first'
}).sort_values(by='units', ascending=False)

print("Productos más vendidos y su calificación:")
print(popularity.head(10))

Productos más vendidos y su calificación:
                           units  product_rating        category
product_name                                                    
Chanel Sauvage              3665             4.0          Beauty
Estée Lauder Chanel No. 5   3439             4.9          Beauty
Chanel Vitamin C            3110             4.3          Beauty
Estée Lauder Vitamin C      2936             4.0          Beauty
Chanel Retinol Cream        2857             4.1          Beauty
Philips K-Elite             2720             4.2  Home & Kitchen
L'Oréal Acqua di Gio        2681             3.6          Beauty
L'Oréal Chanel No. 5        2653             4.1          Beauty
Nike Nuptse                 2575             4.9         Fashion
Estée Lauder Sauvage        2503             4.1          Beauty


In [39]:
# Peso total transportado por cada proveedor (Vendor)
logistica = df_analysis.copy()
logistica['total_weight'] = logistica['units'] * logistica['weight_kg']

vendor_load = logistica.groupby('vendor')['total_weight'].sum().sort_values(ascending=False)

print("Carga Logística Total por Proveedor (kg):")
print(vendor_load)

Carga Logística Total por Proveedor (kg):
vendor
Global Home Solutions    269759.16
Style & Wear Co.         123479.93
TechLogistics Corp       121660.71
Beauty Essentials Ltd    108818.80
Name: total_weight, dtype: float64
